In [5]:
import numpy as np
import torch
import torch.nn as nn
import easydict
import pickle

# 데이터/파라미터 경로 설정 필요
dpath = ""

with open(dpath + '/vocab.pkl', 'rb') as f:
    vocab = pickle.load(f)

In [2]:
args = easydict.EasyDict({    
    'cnn_num_filters': 100,
    'cnn_filter_sizes': [3,4,5],
    'context_size': 35,
    'maxtokens': 30,
    'mlp_hidden': [300,300],
    'dropout': 0.1,
    })

In [16]:
class WORDEBD(nn.Module):
    '''
        An embedding layer that maps the token id into its corresponding word
        embeddings. The word embeddings are kept as fixed once initialized.
    '''
    def __init__(self, vocab, finetune_ebd):#, specific_vocab_size=None):
        super(WORDEBD, self).__init__()

        self.vocab_size, self.embedding_dim = vocab.vectors.size()
        # if specific_vocab_size != None: self.vocab_size = specific_vocab_size
        self.embedding_layer = nn.Embedding(
                self.vocab_size, self.embedding_dim)
        self.embedding_layer.weight.data = vocab.vectors

        self.finetune_ebd = finetune_ebd

        if self.finetune_ebd:
            self.embedding_layer.weight.requires_grad = True
        else:
            self.embedding_layer.weight.requires_grad = False

    def forward(self, data, weights=None):
        '''
            @param text: batch_size * max_text_len
            @return output: batch_size * max_text_len * embedding_dim
        '''
        if (weights is None): #or (self.finetune_ebd == False):
            return self.embedding_layer(data['text'])


class CNNseq(nn.Module):
    '''
        An aggregation method that encodes every document through different
        convolution filters (followed by max-over-time pooling).
    '''
    def __init__(self, ebd, args):
        super(CNNseq, self).__init__()
        self.args = args

        self.ebd = ebd # pre-trained FastText로 initialization된 token representation => WORDEBD => nn.Embedding layer로 매핑된 것

        self.input_dim = self.ebd.embedding_dim

        # Convolution
        self.convs = nn.ModuleList([nn.Conv1d(
                    in_channels=self.input_dim,
                    out_channels=args.cnn_num_filters,
                    kernel_size=K) for K in args.cnn_filter_sizes])
        
        self.relu = nn.ReLU()

        self.ebd_dim = args.cnn_num_filters * len(args.cnn_filter_sizes)

    def _conv_max_pool(self, x, conv_filter=None, weights=None):
        '''
        Compute sentence level convolution
        Input:
            x:      batch_size, max_doc_len, embedding_dim
        Output:     batch_size, num_filters_total
        '''
        assert(len(x.size()) == 3) # [batch_size==max_sentences, max_tokens, embedding_dim]

        x = x.permute(0, 2, 1)  # batch_size, embedding_dim, doc_len
        x = x.contiguous()

        # Apply the 1d conv. Resulting dimension is
        # [batch_size, num_filters, doc_len-filter_size+1] * len(filter_size)
        assert(not ((conv_filter is None) and (weights is None)))
        if conv_filter is not None:
            x = [conv(x) for conv in conv_filter]

        # elif weights is not None:
        #     x = [F.conv1d(x, weight=weights['convs.{}.weight'.format(i)],
        #                 bias=weights['convs.{}.bias'.format(i)])
        #         for i in range(len(self.args.cnn_filter_sizes))]

        ## max pool over time. Resulting dimension is
        ## [batch_size, num_filters] * len(filter_size)
        #x = [F.max_pool1d(sub_x, sub_x.size(2)).squeeze(2) for sub_x in x]
        
        ## nn.MaxPool1d로 다시 생성
        max_pooled_outputs = []
        for sub_x in x:
            pool_size = sub_x.size(2)
            max_pool = nn.MaxPool1d(pool_size) # output shape: [batch_size, num_filters, 1]
            pooled = max_pool(sub_x).squeeze(2) # output shape: [batch_size, num_filters]
            max_pooled_outputs.append(pooled)
        
        # concatenate along all filters. Resulting dimension is
        # output: [batch_size, num_filters_total]
        x = torch.cat(max_pooled_outputs, 1) # output shape: [batch_size, num_filters*3]
        #x = torch.cat(x, 1)
        x = self.relu(x) #F.relu(x)
        return x

    def forward(self, data, weights=None):
        '''
            @param data dictionary
                @key text: batch_size * max_text_len
            @param weights placeholder used for maml

            @return output: batch_size * embedding_dim
        '''

        device = data['text'].device
        
        # Apply the word embedding, result:  batch_size, doc_len, embedding_dim
        
        ebd = self.ebd(data, weights) # ouptut: [batch_size, max_sentences, max_tokens, embedding_dim]

        # apply 1d conv + max pool, result:  batch_size, num_filters_total        
        ref = tuple(data['text'].size())
        shape = (ref[0], ref[1], ( len(self.args.cnn_filter_sizes) * self.args.cnn_num_filters))
        output = torch.randn(shape).to(device)
        
        if weights is None:
            for i in range(ebd.size(0)): # 각 배치에 대해
                out = self._conv_max_pool(ebd[i], conv_filter=self.convs) # 각 문장에 대해 처리 => (35, 300)
                output[i] = out
        
        else:
            for i in range(ebd.size(0)):
                out = self._conv_max_pool(ebd[i], weights=weights)
                output[i] = out
        
        return output


In [19]:
class MLPseq(nn.Module):
    def __init__(self, ebd_dim, args, top_layer=None):
        super(MLPseq, self).__init__()
        
        self.args = args
        self.ebd_dim = ebd_dim

        self.mlp = self._init_mlp(ebd_dim, self.args.mlp_hidden, self.args.dropout)
        self.out = self.get_top_layer(self.args, self.args.n_classes)
        #self.top_layer = top_layer
        self.dropout = nn.Dropout(self.args.dropout)

    @staticmethod
    def get_top_layer(args, n_classes):
        '''
            Creates final layer of desired type
            @return final classification layer
        '''
        return nn.Linear(args.mlp_hidden[-1], n_classes)

        
    def _init_mlp(self, in_d, hidden_ds, drop_rate):
        modules = []

        for d in hidden_ds[:-1]:
            modules.extend([
                nn.Dropout(drop_rate),
                nn.Linear(in_d, d),
                nn.ReLU()])
            in_d = d

        modules.extend([
            nn.Dropout(drop_rate),
            nn.Linear(in_d, hidden_ds[-1])])

        return nn.Sequential(*modules)
    
    def forward(self, XS, YS=None, XQ=None, YQ=None, weights=None, return_preds=False):
        '''
            if y is specified, return loss and accuracy
            otherwise, return the transformed x

            @param: XS: batch_size * input_dim
            @param: YS: batch_size (optional)

            @return: XS: batch_size * output_dim
        '''

        XS = self.mlp(XS)
        XS = self.out(XS) # output: [batch, max_sentence, n_class]
        return XS


## 학습 모델 load 및 inference

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 모델 state path
ebd_model_path = dpath + '/best.ebd'
clf_model_path = dpath + '/best.clf'

ebd_model_params = torch.load(ebd_model_path, map_location=device)
clf_model_params = torch.load(clf_model_path, map_location=device)

# 모델 정의 및 불러온 파라미터 설정
try_model = {}
wordebd = WORDEBD(vocab, finetune_ebd=False)
ebd = CNNseq(wordebd, args).to(device)
try_model['ebd'] = ebd

clf = MLPseq(try_model["ebd"].ebd_dim, args).to(device)
try_model['clf'] = clf

try_model['ebd'].load_state_dict(ebd_model_params)
try_model['clf'].load_state_dict(clf_model_params)

try_model['ebd'].eval()
try_model['clf'].eval()

In [ ]:
# test 데이터 로드

test_np_data = np.load(dpath + '/emotion_test_data.npy')
test_np_label = np.load(dpath + '/emotion_test_label.npy')

test = torch.tensor(test_np_data).to(device)
label = torch.tensor(test_np_label).to(device)

# 모델 inference
out_XS = try_model['clf'](test)